In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import itertools
import random

from utils import *

In [2]:
df = load_process_data('features.csv')
df.head()

,song,bpm,energy,key,chroma_0,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,spectral_centroids
0,ANDUJ - I wanna make you move.mp3,155,-0.349541,G,-1.239201,-0.280945,-0.244045,1.037126,1.061564,0.918715,...,0.872368,0.930172,0.946575,0.508791,0.696535,1.004862,0.572316,0.813991,0.043845,-0.631015
1,Caravel - Untz Untz.mp3,155,0.482397,D#,-0.450468,0.297027,-0.739553,1.637235,1.478732,1.008158,...,-0.180049,-0.617645,0.115161,-0.289649,0.559177,0.557747,0.852728,-0.320232,0.563547,-0.663833
2,Charli xcx - Guess ft. Billie Eilish (Stretch'...,157,1.165463,G#m,0.642735,-0.149614,-0.938502,-0.990076,-1.112110,-0.329308,...,0.677338,-0.246278,-0.543662,-1.304821,-0.494545,-0.491229,0.098825,-0.197423,-0.255321,2.151236
3,CHIHIRO - KALTWEISS REMIX (FREE DL).mp3,160,1.473046,Bm,0.577037,-0.311473,1.298836,0.108297,-1.000726,-0.467201,...,-2.348571,-1.110712,-2.193660,-1.952827,-1.419810,-1.442945,-0.449523,-1.145402,-1.319251,-2.151487
4,Dance With Me - SENTIENT (FREE DL).mp3,160,-0.060338,C#m,0.810482,1.634013,1.757321,0.910389,2.490407,2.088246,...,1.317844,1.094007,0.507446,0.650950,0.045653,0.104884,0.328799,0.330451,0.050199,-0.595636


In [4]:
# sort by energy : 
df = df.sort_values(by='energy', ascending=True)
df

,song,bpm,energy,key,chroma_0,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,spectral_centroids
13,Nyctonian - Purpura Violaceum x Habits - Tove ...,155,-1.915383,F,-0.873528,-1.523328,-0.216757,-1.318294,0.430141,1.532450,...,-1.111507,-1.603026,-1.926670,-1.703820,-1.002815,-1.268665,-1.143505,-1.013165,-0.891244,0.713591
14,Onlynumbers - Euphoric Night Ft. Lucifer � 𝐅𝐑𝐄...,158,-1.647995,Am,1.055058,-0.841117,-0.220996,0.017659,-1.210021,-1.050165,...,-0.485125,-0.718986,0.012185,-0.389164,-0.660823,-0.805967,-1.006429,-0.632932,-0.175757,0.153680
19,Save Me (RiVid Remix).mp3,155,-1.517100,A,0.727477,2.263316,1.524302,-0.360769,1.031670,-0.120188,...,0.413739,-0.181840,0.157713,0.167372,-0.178383,-1.491167,-0.756345,0.274128,1.219291,1.570501
21,Wave (KomaCasper Remix) [feat. REDCHINAWAVE].mp3,170,-1.511540,Fm,-0.305984,-0.763454,-1.678419,-1.282339,-0.062162,-1.827762,...,-0.060193,2.281751,2.318097,0.945356,-0.607260,-0.749067,-3.101785,-2.395507,1.668423,-0.621516
15,Premiere� AREA ØNE - Balance [Free DL].mp3,160,-0.887784,Em,-2.230572,-1.881471,-0.135998,-0.383242,-1.520933,-1.719127,...,-0.657798,-0.555145,-0.922867,0.689195,-0.523625,1.312186,1.442702,-0.325412,-1.941149,-1.269265
9,Love Game (Black Traffic Remix).mp3,154,-0.844180,B,1.318419,0.110310,-0.782087,-0.153992,-0.934868,0.272421,...,-0.349725,-0.240720,-1.009234,-0.986471,-0.515973,-0.112472,0.036283,0.278150,-0.743435,-0.585158
12,"Nicolas Julian, NOTMYTYPE, AENJAY & SP!NNER - ...",158,-0.543669,D#,-0.112842,-0.012177,0.991522,2.423085,0.865024,-0.616187,...,-0.427012,-0.370767,-0.561686,-0.391764,-1.209439,-0.826505,-0.554616,-0.318889,-0.202389,-1.021695
7,Frankyeffe - Touch Me (Caravel Remix).mp3,155,-0.372905,D,0.415000,1.378591,1.729604,0.726282,-0.254842,-0.248509,...,-0.573431,0.033244,-0.225580,0.661004,-0.509168,-1.317193,-0.700507,-0.213156,0.071645,-0.029095
0,ANDUJ - I wanna make you move.mp3,155,-0.349541,G,-1.239201,-0.280945,-0.244045,1.037126,1.061564,0.918715,...,0.872368,0.930172,0.946575,0.508791,0.696535,1.004862,0.572316,0.813991,0.043845,-0.631015
8,Free DL � KE�TR - La Classe (Jul Le Sang Edit)...,155,-0.220420,G,0.183904,0.432441,0.626981,-0.540007,-0.947545,-0.192979,...,-1.606833,-0.935347,-0.542117,0.171903,0.172018,1.485167,0.673321,0.461938,0.538190,-0.209346


In [6]:
names = df['song']
X = df.drop(['song', 'bpm', 'key'], axis=1).values

In [9]:
# PCA to plot the data in 2D (using plotly)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Keep a column with color that change with bpm increasing (bpm being the first column of X)
X_pca = np.c_[X_pca, df['bpm'].values]

# same but hovering over the points to see the song name
fig = go.Figure()
for i in range(len(X_pca)):
    fig.add_trace(go.Scatter(x=[X_pca[i, 0]], y=[X_pca[i, 1]], mode='markers', marker=dict(color=X_pca[i, 2]), text=names[i], name=names[i], showlegend=False))
fig.show()



In [10]:
# 'Pier - Angèle Saiyan (Techno Edit).mp3" bpm is too low, we remove it
df = df[df.song != 'Pier - Angèle Saiyan (Techno Edit).mp3']

# Assuming df is your DataFrame with songs and their BPMs
G = create_graph(df)
G = create_graph_key_constraint(df)

# Normalize BPM for color mapping
bpm_normalized = (df.set_index('song')['bpm'] - df['bpm'].min()) / (df['bpm'].max() - df['bpm'].min())

# Calculate node positions
pos = nx.spring_layout(G)

# Node trace
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=10,
        color=[bpm_normalized[node] for node in G.nodes()],
        colorscale='Viridis',
        colorbar=dict(title='BPM'),
        line_width=2,
        opacity=0.8  # Reduced opacity
    ),
    text=[node for node in G.nodes()],  # Hover text
    hoverinfo='text'
)

# Edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Song Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

In [ ]:
def find_shortest_hamiltonian_path(G):
    # Store the shortest path and its length
    shortest_path = None
    shortest_path_length = float('inf')

    # Iterate over all permutations of nodes to check each possible path
    for path in itertools.permutations(G.nodes()):
        current_length = 0
        valid_path = True

        # Calculate the length of the current path
        for i in range(len(path) - 1):
            if G.has_edge(path[i], path[i + 1]):
                current_length += G[path[i]][path[i + 1]]['weight']
            else:
                valid_path = False
                break

        # Update the shortest path if a shorter one is found
        if valid_path and current_length < shortest_path_length:
            shortest_path_length = current_length
            shortest_path = path

        print(f"Checked path: {path}, Length: {current_length}")

    return shortest_path, shortest_path_length


path, length = find_shortest_hamiltonian_path(G)
print(f"Shortest Hamiltonian path: {path}, Length: {length}")